In [1]:
import numpy as np
import os
import subprocess
from contextlib import contextmanager

First, we'll read in the files created by the Fortran HF code. Things are a little messy thanks to the way I stored things in the past, but they are standard numpy arrays after this cell. Take note of the ordering and suggest a more agreeable choice if you've got an opinion.

In [57]:
def read_wfs(restartfile):
    #restartfile = '../HF/wf_numpy.bin'

    intchunk = np.dtype(np.int32).itemsize
    realchunk = np.dtype(np.float64).itemsize
    h = np.fromfile(restartfile, dtype=np.float64, count=1, offset=0)[0]
    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+2*intchunk))[0]
#     print("hbox = ", h)
#     print("nbox = ",nbox)
#     print("lmax = ",lmax)
#     print("nmax = ",nmax)

    #sortstates, sortenergies, wfr,
    sortstates = np.fromfile(restartfile, dtype=np.int32, count=nmax*3*2, offset=(realchunk+3*intchunk)).reshape(2,3,nmax)
    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(realchunk+3*intchunk+nmax*3*2*intchunk)).reshape(2,nmax)
    wfs = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(realchunk+3*intchunk+nmax*3*2*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return h,nbox,lmax,nmax,sortstates,sortenergies,wfs

def print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wfs):
    with open(restartfile, 'wb') as file:
        file.write(h)
        file.write(nbox)
        file.write(lmax)
        file.write(nmax)
        file.write(sortstates.reshape((nmax*3*2,)))
        file.write(sortenergies.reshape((nmax*2,)))
        file.write(wfs.reshape((2*2*(lmax+1)*lmax*(nbox+1),)))

    return

def read_hpsi(restartfile):

    intchunk = np.dtype(np.int32).itemsize

    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(2*intchunk))[0]

    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(3*intchunk)).reshape(2,nmax)
    hpsi = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(3*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return sortenergies,hpsi

def get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,wf):
    restartfile = '../HF/wf_numpy.bin'
    print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wf)
    make_input(params[i],1)
    with cwd('../HF/'):
        errors.append(run_HF("None",1))
    restartfile = '../HF/hpsi.bin'
    sortenergies,hpsi = read_hpsi(restartfile)

    return hpsi

In [44]:
@contextmanager
def cwd(path):
    oldpwd=os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)

def spin(spin):
    if(spin==1): return -0.5
    return 0.5

def make_input(params,restart):
    in_template = "../HF/in.template"
    f1 = open(in_template, 'r')
    
    int_string = "&interaction t0= "+str(params[0])+",x0=0.77600,t1=484.2300,x1=-0.31700,t2=-556.6900,x2=-1.0,t3=13757.0,x3=1.2630,w0=125.0,sig=6.0,icoul=1,icm=1,j2terms=.true. / \n"
    if(restart):
        io_string = "&io printwf=F,printdens=F,restartwf=T,printhpsi=T /"
    else:
        io_string = "&io printwf=T,printdens=F,restartwf=F,printhpsi=F /"
    in_out = "../HF/in"
    f2 = open(in_out, 'w')
    f1.seek(0)
    f2.write(f1.read())
    f2.write(int_string)
    f2.write(io_string)
    f2.write("\n")

    f1.close()
    f2.close()

    return

def run_HF(tag,restart):
    noNaN = 1
    if(restart == 0):
        os.system("cp in results/in."+tag)
        #print(os.system("../HF/./main"))
        output = subprocess.run(["./main"], capture_output=True)
        #print(output)
        #print(subprocess.run("../HF/./main",shell=True))
        os.system("mv out results/out."+tag)
        os.system("mv wf_numpy.bin results/wf_numpy.bin."+tag)
        noNaN = os.system("grep -q NaN results/out."+tag)
    else:
        output = subprocess.run(["./main"], capture_output=True)
        #os.system("mv out results/out_rst."+tag)
        #os.system("mv hpsi.bin results/hpsi.bin."+tag)
        #print(output)
        #noNaN = os.system("grep -q NaN results/out_rst."+tag)
    if (noNaN == 0):
        print("NaNs in run {tag}!".format(tag=tag))
        return 1

    return 0


The following cell defines a list of parameters (only one right now) and then loops over those parameters and runs the HF code to generate outputs and WFs.

The inputs, outputs, and WFs are copied to a results folder (this can be changed) with some tag appended to the file name. In the future we may want to change how this tag is generated.

In [10]:
params = [[-2483.45],[-2485.45],[-2453.45],[-2583.45],[-1983.45],]

errors = []

for i in range(len(params)):
    make_input(params[i],0)
    with cwd('../HF/'):
        errors.append(run_HF(str(i),0))




NaNs in run 4!


Now we read the files back in and print the energies of the single particle states to see whats up.

In [27]:
for i in range(len(params)):
    if(errors[i] == 1): continue
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)
    print(sortenergies[sortenergies < 0.0])


#h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs()
# npart = 0
# for i in range(nmax):
#     if(sortstates[0,0,i]>0):
#         nfill = int(2*(sortstates[0,1,i]+spin(sortstates[0,2,i])) + 1)
#         if (sortstates[0,1,i] == 0): nfill = 2
#         npart += nfill
#         print(npart)

[-49.68895404 -35.89084008 -33.84649678 -22.12395077 -17.72178801
 -17.42934489  -9.23136256 -45.87413675 -34.25177677 -30.4309684
 -22.40003827 -16.74195047 -15.31277247]
[-49.95249716 -36.09015803 -34.02871369 -22.26319554 -17.83170985
 -17.53014294  -9.3178674  -46.1503477  -34.47515512 -30.62320195
 -22.57424528 -16.8790529  -15.42882466]
[-45.84523083 -32.98656995 -31.18665098 -20.10136506 -16.12312307
 -15.95788421  -7.98626695 -41.85989697 -31.01135766 -27.63739743
 -19.8828084  -14.75988052 -13.62980033]
[-64.04105666 -46.77671843 -43.74722209 -29.79788411 -23.74954255
 -22.89918174 -14.12217506 -61.07572786 -46.61419015 -41.01603689
 -32.15028024 -24.39412814 -21.73595017]


In [55]:
for i in range(len(params)-4):
    if(errors[i] == 1): continue
    #restartfile = '../HF/wf_numpy.bin.'+str(i)
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)
    restartfile = '../HF/wf_numpy.bin'
    print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wf)
    make_input(params[i],1)
    with cwd('../HF/'):
        errors.append(run_HF(str(i),1))
    restartfile = '../HF/results/hpsi.bin.'+str(i)
    sortenergies,hpsi = read_hpsi(restartfile)
    part = 0
    i_s = sortstates[0,0,part]
    i_l = sortstates[0,1,part]
    i_n = sortstates[0,2,part]-1
    print(i_n,i_l,i_s)
    print(np.sum(-h*wf[0,i_s,i_l,i_n,:]*hpsi[0,i_s,i_l,i_n,:]))
    #ir,sortstates(i,1,iq),sortstates(i,2,iq),sortstates(i,3,iq),iq


0 0 1
-49.71129278586508


Now we can calculate the hpsi (or, G) objects from Wfs. Here I am reading in training Wfs just as a test.

In [94]:
for i in range(len(params)-4):
    if(errors[i] == 1): continue
    # The following is to simply read in the trial WFs during testing
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)

    # get hpsi
    hpsi = get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,wf)

    #Calculate energies as a test for levels
    occupied = 0
    for i in range(nmax):
        isospin = 0
        level = i
        i_s = sortstates[isospin,2,level]-1
        i_l = sortstates[isospin,1,level]
        i_n = sortstates[isospin,0,level]-1
        if(i_s < 0): continue
        occupied += 1
        print("Level: {}, N: {}, L: {}, S: {}".format(level+1,i_n+1,i_l,i_s))
        print("Energy: {}".format(np.sum(-h*wf[isospin,i_s,i_l,i_n,:]*hpsi[isospin,i_s,i_l,i_n,:])))

print("There are {} occupied levels!".format(occupied))

Level: 1, N: 1, L: 0, S: 0
Energy: -49.71129278586775
Level: 2, N: 1, L: 1, S: 1
Energy: -35.89095593307716
Level: 3, N: 1, L: 1, S: 0
Energy: -33.846486193249426
Level: 4, N: 1, L: 2, S: 1
Energy: -22.12394236411225
Level: 5, N: 2, L: 0, S: 0
Energy: -17.757508449901596
Level: 6, N: 1, L: 2, S: 0
Energy: -17.429233705493058
Level: 7, N: 1, L: 3, S: 1
Energy: -9.231279254081628
There are 7 occupied levels!
